# Storytelling: An example of runtime optimization in Python
## by Abram Hindle < hindle1@ualberta.ca >

* A long time ago a composer named Edward Elgar wrote a postcard to his girl friend. In the bottom corner of the postcard was a cryptic message.

* Many cryptographers have tried to decode it as English. 
   * We found that that decipherments of the characters do not match English distributions of characters. 
   * We found better evidence it matched music. So we decided to come up with decipherments that were musical.



![Dorabella Cipher](Dorabella-cipher-image.png)

Our idea is the direction and bumps are meaningful so we make a symbolic encoding where direction and bumps are encoded.

⇐2 ⇒3 ⇖2 ⇐3 ⇐1 ⇑2 ⇓1 ⇐3 ⇗1 ⇙2 ⇖3 ⇘2 ⇘1 ⇖1 ⇘2 ⇑3 ⇘2 ⇘2 ⇑2 ⇒3 ⇒3 ⇘2 ⇖1 ⇙1 ⇙2 ⇙1 ⇑1 ⇖3 ⇘3 

⇓1 ⇘2 ⇓1 ⇑2 ⇙1 ⇐3 ⇗1 ⇗2 ⇐3 ⇖2 ⇘2 ⇘2 ⇖2 ⇑2 ⇖1 ⇘1 ⇓1 ⇘2 ⇖3 ⇘2 ⇑2 ⇓2 ⇘3 ⇘1 ⇖1 ⇙1 ⇗1 ⇗1 ⇙1 ⇖3 ⇘3 

⇖2 ⇘3 ⇑2 ⇓2 ⇘3 ⇖2 ⇖1 ⇓2 ⇓3 ⇑1 ⇘3 ⇖2 ⇘2 ⇑2 ⇓2 ⇘1 ⇘3 ⇖1 ⇐3 ⇒3 ⇑1 ⇘3 ⇑2 ⇐3 ⇖1 ⇙1 ⇐3

Here's our program. 

* It generates a mapping of symbols to notes
* The mapping must be in key (e.g. C-minor)
* Converts a document of symbols into notes. 
* Writes out the decrypted document to disk

It's too slow.

# Rules for Optimization

1. Don't do it.
2. Only do it if you can measure the quality you want to optimize
3. Don't do it yet.
4. Only do it if you record your original performance.
5. Make sure you're optimizing what you want optimized
6. Record original performance
7. Investigate performance
8. Change system
9. Measure new performance
10. Compare
11. If not enough goto 6.

# Knuth's views on the subject 

From C2 Wiki http://wiki.c2.com/?PrematureOptimization
    
> /Premature optimization is the root of all evil/ -- DonaldKnuth
> 
> In DonaldKnuth's paper "Structured Programming With Goto Statements", he wrote:
> > "Programmers waste enormous amounts of time thinking about, or worrying about, the speed of noncritical parts of their programs, and these attempts at efficiency actually have a strong negative impact when debugging and maintenance are considered. We should forget about small efficiencies, say about 97% of the time: premature optimization is the root of all evil. Yet we should not pass up our opportunities in that critical 3%." 

We're not engaging in premature optimization. We have a legitimate runtime problem.

In [1]:
# Make sure the output dir is available
!mkdir abram.gen.out

mkdir: cannot create directory ‘abram.gen.out’: File exists


In [2]:
# -*- coding: utf-8 -*-
# UTF8 hacks invoked.
import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')
#reload(sys)
import codecs
#UTF8Writer = codecs.getwriter('utf8')
#sys.stdout = UTF8Writer(sys.stdout)
import util
import argparse
import inkey
import music21
import copy
import itertools
import re
import json
import time

n_dirs = 8
n_bumps = 3
directions = {
    "⇑":0,
    "⇗":1,
    "⇒":2,
    "⇘":3,
    "⇓":4,
    "⇙":5,
    "⇐":6,
    "⇖":7
}
inverse_directions = dict([(directions[key],key) for key in directions])
dir_string = "".join([inverse_directions[x] for x in range(0,n_dirs)])

def token_to_dir_tuple(token):
    return (directions[token[0:-1]], int(token[-1]))

def dir_tuple_to_string(token):
    return inverse_directions[token[0]]+str(token[1])

def load_transcript(filename):
    tokens = util.load_split_file(filename)
    return [token_to_dir_tuple(token) for token in tokens]

def rotate_list(l):
    return l[1:] + [l[0]]

def n_rotated_lists(l,n):
    out = range(0,n)
    out[0] = l
    for i in range(1,n):
        out[i] = rotate_list(out[i-1])
    return out

full_note      = 4.0
half_note      = 2.0
quarter_note   = 1.0
eighth_note    = 0.5
sixteenth_note = 0.25
note_durations = [full_note, half_note, quarter_note, eighth_note, sixteenth_note]

def make_note( midi21_pitch, duration ):
    new_note = music21.note.Note(midi21_pitch)
    new_note.duration = music21.duration.Duration(duration) # is this right
    return new_note

def pitches_of_key(mkey):
    m = re.match('([A-G][-#]?)(major|minor)',mkey)
    k = m.group(1)
    maj = m.group(2)
    our_key = music21.key.Key(k,maj)
    our_pitches = our_key.pitches
    return our_pitches

def generate_abram_song(song_tuples, mkey, note_mapping, bump_mapping, octave):
    """ returns a list of music21 notes """
    our_pitches = pitches_of_key(mkey)
    # print(our_pitches)
    # modify the key to the appropriate octave
    our_key_notes = [inkey.note_octave_mod(note, octave-4) for note in our_pitches]
    # now get the note mapping to notes
    our_notes = [our_key_notes[x] for x in note_mapping]
    # process the tuple
    # print song_tuples
    # print bump_mapping
    assert sum([len(t)==2 for t in song_tuples]) == len(song_tuples)
    # print [ (our_notes[n], bump_mapping[b-1]) for n,b in song_tuples ]
    return [ make_note(our_notes[n], bump_mapping[b-1]) for n,b in song_tuples ]

def note_to_str(note):
    return note.fullName + " " + str(note.duration) #music21.Note.fullName(note)
def song_to_txt(song):
    return "\n".join([note_to_str(note) for note in song])

def save_textfile(txt, filename):
    fd = file(filename,"w")
    fd.write(txt)
    fd.close()

def song_to_mid(song):
    stream = music21.stream.Stream()
    for elm in song:
        stream.append(elm)
    return stream
    # mf = music21.midi.translate.streamToMidiFile(stream)

def save_midifile(midi, midi_file):
    fp = midi.write('midi', fp=midi_file)
    
def strify(l):
    return [str(x) for x in l]
    
def generate_and_save_song(song_tuples,mkey, note_mapping, bump_mapping,octave):
    song = generate_abram_song(song_tuples, mkey, note_mapping, bump_mapping, octave)
    file_name = "abram.gen.out/" + mkey + ".O." + str(octave) + ".N." + ".".join("".join(strify(note_mapping))) + ".T." + "_".join(strify(bump_mapping))
    txt_file_name = file_name + ".txt"
    mid_file_name = file_name + ".mid"
    txt = song_to_txt(song)
    save_textfile(txt, txt_file_name)
    # print(song)
    mid = song_to_mid(song)
    save_midifile(mid, mid_file_name)

__grid_search_print__ = True
def grid_search(params,param_keys,song_tuples,f=None,maxtime=None):
    if maxtime is None:
        maxtime = float("inf")
    start = time.time()
    iters = 0
    for t in itertools.product(*[params[x] for x in param_keys]):
        if __grid_search_print__:
            print(t)
        f(song_tuples,*t)
        iters += 1
        if (time.time() - start > maxtime):
            print("Times up! %s" % iters)
            break
    return iters

def my_main(maxtime=10):
    filename = "abram-dirs.txt"
    song_tuples = load_transcript(filename)
    print(inkey.get_all_keys().keys())
    #print("\t".join([dir_tuple_to_string(x) for x in our_tokens]))
    params = {
        "keys":inkey.get_all_keys().keys(),
        # permutations of indices
        #  could do all permutations too
        "dir_notes":n_rotated_lists(range(0,n_dirs),n_dirs),
        # "dir_notes":list(itertools.permutations(range(0,n_dirs),n_dirs)) # if you want 40k combos
        # map bumps to duration
        "bump_durations":list(itertools.permutations(note_durations,n_bumps)),
        "octaves":range(3,6),
    }
    return grid_search(params,["keys","dir_notes","bump_durations","octaves"], 
                song_tuples, f=generate_and_save_song, maxtime=maxtime)

def generate_and_estimate_song(estimate,song_tuples,mkey, note_mapping, bump_mapping,octave):
    song = generate_abram_song(song_tuples, mkey, note_mapping, bump_mapping, octave)
    file_name = "abram.gen.out/" + mkey + ".O." + str(octave) + ".N." + ".".join("".join(strify(note_mapping))) + ".T." + "_".join(strify(bump_mapping))
    txt = song_to_text(song)
    return estimate(txt), file_name
    
def my_estimate_all(estimator, transcription_song_tuples):
    estimates = dict()
    def helper(estimator,song_tuples,mkey, note_mapping, bump_mapping,octave):
        estimate, name = generate_and_estimate_song(estimator,song_tuples,mkey, note_mapping, bump_mapping,octave)
        estimates[name] = estimate
        print((estimate, name))
    
    params = {
        "keys":inkey.get_all_keys().keys(),
        # permutations of indices
        #  could do all permutations too
        # "dir_notes":n_rotated_lists(range(0,n_dirs),n_dirs),
        "dir_notes":list(itertools.permutations(range(0,n_dirs),n_dirs)), # if you want 40k combos
        # map bumps to duration
        "bump_durations":list(itertools.permutations(note_durations,n_bumps)),
        "octaves":range(3,6),
    }
    grid_search(params,["keys","dir_notes","bump_durations","octaves"], song_tuples, f=helper)
    pickle.dump(estimates, file('abram-song-estimates.pkl','w'))    

def my_estimate(args):
    filename = args.transcript
    song_tuples = load_transcript(filename)
    my_estimate_all(None,song_tuples)
    
def test_rotate_list():
    assert rotate_list([0,1,2,3]) == [1,2,3,0]
    assert rotate_list([0,1,2,3,4]) == [1,2,3,4,0]
    assert rotate_list(range(0,100))[0:-1] == range(1,100)
    input  = [0,1,2]
    output = [[0,1,2],[1,2,0],[2,0,1]]
    assert json.dumps(n_rotated_lists(input, 3)) == json.dumps(output)    

def test_notes():
    note = music21.pitch.Pitch("C#4") 
    new_note = make_note( note, quarter_note )
    assert new_note.duration.type == 'quarter'
    assert new_note.pitch.fullName == 'C-sharp in octave 4'    
    song = song_to_mid([copy.deepcopy(new_note) for i in range(0,10)])
    notes = list(song.notesAndRests)
    for i in range(0,len(notes)):
        assert notes[i].duration.type == new_note.duration.type
        assert notes[i].pitch.fullName == new_note.pitch.fullName
    cmajor = pitches_of_key("Cmajor")
    assert [x.name for x in cmajor] == ["C","D","E","F","G","A","B","C"]
    
def my_tests(args):
    test_rotate_list()
    test_notes()
    


In [3]:
# run our main for 5 seconds
my_main(5)


['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 1.0), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.5), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 2.0, 0.25), 5)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 3)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0), 4)
('Emajor', [0, 1, 2, 3, 4, 5, 6, 7], (4.0, 1.0, 2.0),

77

In [4]:
# Let's measure performance!
__grid_search_print__ = False # disable debugging
seconds = 1  # measure for 1 second
repeats = 10 # measure 10 times
def mean(x):
    return sum(x)/len(x)
perf = map(lambda x: my_main(seconds),range(0,repeats))
perf

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 18
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 18
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 18
['Emajor',

[18, 18, 18, 18, 18, 18, 18, 18, 18, 18]

In [5]:
print("Current Performance ~ %s iters per %s seconds " % (mean(perf), seconds))

Current Performance ~ 18 iters per 1 seconds 


4. Original performance? 
   * 14-18 songs per second? 
   * That's really very poor.
5. Make sure we are optimizing what want optimized? 
   * Time.
   * But at least we know what we want: More iters per second.
6. Record original performance
   * 14-18 song per second
7. Investigate performance
   * We're going to use a profiler
8. Change system
   * not until we profile!

At this point we either have a good idea of what is slow or we need some help to find out what.

Profilers instrument running code and take statistics about the number of calls and the 

Checkout https://docs.python.org/2/library/profile.html

In [6]:
import cProfile

In [7]:
# Instrument my program and run my_main for 4 seconds
cProfile.run('my_main(4)','cpout.txt')

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 39


## FYI Performance decreased due to instrumentation

In [8]:
import pstats

In [9]:
# Let's summarize the profiling run
p = pstats.Stats('cpout.txt')
p.strip_dirs().sort_stats(-1).print_stats() # order by name

Mon May 13 13:45:03 2019    cpout.txt

         5354904 function calls (5224410 primitive calls) in 4.072 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       39    0.002    0.000    0.104    0.003 <ipython-input-2-02a0ed9aa61e>:100(song_to_mid)
       39    0.000    0.000    2.115    0.054 <ipython-input-2-02a0ed9aa61e>:107(save_midifile)
       78    0.000    0.000    0.000    0.000 <ipython-input-2-02a0ed9aa61e>:110(strify)
       39    0.001    0.000    4.073    0.104 <ipython-input-2-02a0ed9aa61e>:113(generate_and_save_song)
        1    0.000    0.000    4.074    4.074 <ipython-input-2-02a0ed9aa61e>:125(grid_search)
        1    0.000    0.000    4.074    4.074 <ipython-input-2-02a0ed9aa61e>:140(my_main)
       87    0.000    0.000    0.000    0.000 <ipython-input-2-02a0ed9aa61e>:35(token_to_dir_tuple)
        1    0.000    0.000    0.000    0.000 <ipython-input-2-02a0ed9aa61e>:41(load_transcript)
        7    0.000

       78    0.000    0.000    0.030    0.000 iterator.py:1309(__init__)
       78    0.000    0.000    0.000    0.000 iterator.py:1340(reset)
     6864    0.014    0.000    0.019    0.000 iterator.py:1349(__next__)
    10491    0.030    0.000    0.091    0.000 iterator.py:135(__next__)
     6786    0.003    0.000    0.003    0.000 iterator.py:1439(iteratorStack)
     6786    0.009    0.000    0.025    0.000 iterator.py:1482(currentHierarchyOffset)
      117    0.000    0.000    0.025    0.000 iterator.py:351(__bool__)
    17706    0.009    0.000    0.009    0.000 iterator.py:407(updateActiveInformation)
      390    0.000    0.000    0.001    0.000 iterator.py:421(reset)
      195    0.000    0.000    0.000    0.000 iterator.py:433(resetCaches)
      390    0.000    0.000    0.000    0.000 iterator.py:443(cleanup)
       78    0.001    0.000    0.031    0.000 iterator.py:473(matchingElements)
    30537    0.013    0.000    0.034    0.000 iterator.py:528(matchesFilters)
       78    0.

<pstats.Stats instance at 0x7fabdc85add0>

In [10]:
# Print by cumulative time
# That is the sum of time for all calls and child calls
# These are functions who call on others to do work
p.sort_stats('cumulative').print_stats(10)

Mon May 13 13:45:03 2019    cpout.txt

         5354904 function calls (5224410 primitive calls) in 4.072 seconds

   Ordered by: cumulative time
   List reduced from 421 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.074    4.074 <string>:1(<module>)
        1    0.000    0.000    4.074    4.074 <ipython-input-2-02a0ed9aa61e>:140(my_main)
        1    0.000    0.000    4.074    4.074 <ipython-input-2-02a0ed9aa61e>:125(grid_search)
       39    0.001    0.000    4.073    0.104 <ipython-input-2-02a0ed9aa61e>:113(generate_and_save_song)
       39    0.000    0.000    2.115    0.054 <ipython-input-2-02a0ed9aa61e>:107(save_midifile)
       39    0.000    0.000    2.114    0.054 __init__.py:246(write)
       39    0.001    0.000    2.114    0.054 base.py:2529(write)
       39    0.000    0.000    2.103    0.054 subConverters.py:921(write)
       39    0.000    0.000    2.045    0.052 translate.py:237(mu

<pstats.Stats instance at 0x7fabdc85add0>

In [11]:
# total time is the time WITHIN the funciton but not subcalls
# these are the beefy functions where time is spent
p.sort_stats('tottime').print_stats(10)

Mon May 13 13:45:03 2019    cpout.txt

         5354904 function calls (5224410 primitive calls) in 4.072 seconds

   Ordered by: internal time
   List reduced from 421 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    30147    0.193    0.000    0.524    0.000 collections.py:50(__init__)
123084/2925    0.165    0.000    1.502    0.001 copy.py:145(deepcopy)
    30147    0.131    0.000    0.288    0.000 _abcoll.py:548(update)
     3432    0.115    0.000    1.029    0.000 base.py:479(_deepcopySubclassable)
     7878    0.094    0.000    0.135    0.000 pitch.py:2309(_setName)
    25974    0.087    0.000    0.597    0.000 base.py:409(__init__)
    29406    0.070    0.000    0.590    0.000 sites.py:167(__init__)
    20358    0.069    0.000    0.135    0.000 sites.py:330(add)
    26598    0.069    0.000    0.215    0.000 pitch.py:2037(_getPs)
    54873    0.065    0.000    0.065    0.000 collections.py:71(__setitem__)




<pstats.Stats instance at 0x7fabdc85add0>

# Looking at the results

Above, do you notice what library is taking up most of the top 6 places?

2 of those calls are to init, so that's object creation.

There's also a reference to deepcopy.

In [12]:
# this is the number of times a function is called
# speeding these up just a little bit can change
# their total time drasticly
p.sort_stats('ncalls').print_stats(20)

Mon May 13 13:45:03 2019    cpout.txt

         5354904 function calls (5224410 primitive calls) in 4.072 seconds

   Ordered by: call count
   List reduced from 421 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   694980    0.045    0.000    0.045    0.000 {id}
   232050    0.047    0.000    0.049    0.000 {getattr}
   204064    0.053    0.000    0.132    0.000 {isinstance}
   195117    0.022    0.000    0.022    0.000 {method 'append' of 'list' objects}
   186810    0.027    0.000    0.027    0.000 {method 'get' of 'dict' objects}
   137631    0.028    0.000    0.029    0.000 {setattr}
129792/129714    0.014    0.000    0.014    0.000 {len}
123084/2925    0.165    0.000    1.502    0.001 copy.py:145(deepcopy)
    96798    0.020    0.000    0.020    0.000 {round}
    88959    0.039    0.000    0.039    0.000 _weakrefset.py:70(__contains__)
    83550    0.035    0.000    0.051    0.000 weakrefTools.py:49(unwrapWeakref)
    82485 

<pstats.Stats instance at 0x7fabdc85add0>

# Cost of creation?

I am concerned about these calls to music21

      968    0.122    0.000    1.007    0.001 base.py:479(_deepcopySubclassable)
     2222    0.101    0.000    0.147    0.000 pitch.py:2309(_setName)
     7326    0.095    0.000    0.594    0.000 base.py:409(__init__)

In [13]:
# here's a candidate function that creates lots of music21 objects.

def make_note( midi21_pitch, duration ):
    new_note = music21.note.Note(midi21_pitch)
    new_note.duration = music21.duration.Duration(duration) # is this right
    return new_note


# Problem: Makes too many objects that are for some reason very slow to create.
    
These are notes, with set duration. Luckily how I use these notes is immutable. I do not change them.

Therefore I can SHARE notes if they are already created. That is I can memoize the output of this function!

In [14]:
# keep the old code to really make it
def _make_note( midi21_pitch, duration ):
    new_note = music21.note.Note(midi21_pitch)
    new_note.duration = music21.duration.Duration(duration) # is this right
    return new_note

# a place to cache the created notes
__note_cache__ = dict()
# memoize
def make_note( midi21_pitch, duration ):
    # make a key
    note_key = str(midi21_pitch)+str(duration)
    # if the note isn't in the cache make it
    if not note_key in __note_cache__:
        __note_cache__[note_key] = _make_note(midi21_pitch, duration)
    return __note_cache__[note_key]

In [15]:
# let's make this more repeatable
def bench(seconds=1,repeats=10):
    perf = map(lambda x: my_main(seconds),range(0,repeats))
    print("Current Performance ~ %s iters per %s seconds " % (mean(perf), seconds))
    print(perf)
    return perf

In [16]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']


StreamException: the object (<music21.note.Note D#>, id()=140376110520400) is already found in this Stream (<music21.stream.Stream 0x7fabdfb0ee10>, id()=140376169049616)

# Ok when I was doing this I didn't have this problem

The music21 library has changed, so I guess so should my presentation of my problem. *sigh*

## Performance is not stable between revisions!

In [17]:
def song_to_mid(song):
    stream = music21.stream.Stream()
    for elm in song:
        note = music21.note.Note()
        note.pitches = elm.pitches
        note.duration = elm.duration
        stream.append(note)
    return stream

In [18]:
# run it
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 18
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 18
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 19
['Emajor',

[18, 18, 19, 19, 18, 18, 18, 17, 18, 17]

It doesn't look like performance improved very much :(

In [19]:
# profile it
cProfile.run('my_main(4)','cpout2.txt')


['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 44


In [20]:
# investigate the profile
p = pstats.Stats('cpout2.txt')
p.sort_stats('tottime').print_stats(10)
p.sort_stats('cumtime').print_stats(20)


Mon May 13 13:45:42 2019    cpout2.txt

         5806248 function calls (5665624 primitive calls) in 4.039 seconds

   Ordered by: internal time
   List reduced from 426 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
139900/3300    0.170    0.000    1.485    0.000 /usr/lib/python2.7/copy.py:145(deepcopy)
    34022    0.145    0.000    0.490    0.000 /usr/lib/python2.7/collections.py:50(__init__)
    12716    0.137    0.000    0.198    0.000 /home/hindle1/.local/lib/python2.7/site-packages/music21/pitch.py:2309(_setName)
    34022    0.133    0.000    0.300    0.000 /usr/lib/python2.7/_abcoll.py:548(update)
     3872    0.120    0.000    0.990    0.000 /home/hindle1/.local/lib/python2.7/site-packages/music21/base.py:479(_deepcopySubclassable)
    29314    0.091    0.000    0.620    0.000 /home/hindle1/.local/lib/python2.7/site-packages/music21/base.py:409(__init__)
    90479    0.074    0.000    0.074    0.000 /home/hindle1/.local/

<pstats.Stats instance at 0x7fabdd064290>

I saw this:
    
    40    0.000    0.000    1.475    0.037 <ipython-input-20-50c167bfa7f6>:67(pitches_of_key)
    
Very few calls. The function doesn't do much but it does expensive things. So maybe we can memoize or refactor it?

In [21]:
__pitches_of_key__ = dict()
def _pitches_of_key(mkey):
    m = re.match('([A-G][-#]?)(major|minor)',mkey)
    k = m.group(1)
    maj = m.group(2)
    our_key = music21.key.Key(k,maj)
    our_pitches = our_key.pitches
    return our_pitches

# memoize
def pitches_of_key(mkey):
    if not mkey in __pitches_of_key__:
        __pitches_of_key__[mkey] = _pitches_of_key(mkey)
    return __pitches_of_key__[mkey]



In [22]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 26
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 28
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 26
['Emajor',

[26, 28, 26, 28, 29, 28, 29, 26, 24, 29]

Boom. Performance Improved! 14 to 23. Good. 33% faster

In [23]:
cProfile.run('my_main(4)','cpout3.txt')
p = pstats.Stats('cpout3.txt')
p.sort_stats('tottime').print_stats(10)
p.sort_stats('cumtime').print_stats(20)


['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 63
Mon May 13 13:46:07 2019    cpout3.txt

         6097065 function calls (5914437 primitive calls) in 4.016 seconds

   Ordered by: internal time
   List reduced from 332 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
177645/630    0.206    0.000    2.067    0.003 /usr/lib/python2.7/copy.py:145(deepcopy)
     5544    0.176    0.000    1.511    0.000 /home/hindle1/.local/lib/python2.7/site-packages/music21/base.py:479(_deepcopySubclassable)
    10962    0.122    0.000    0.180    0.000 /home/hindle1/.local/lib/python2.7/site-packages/music21/pitch.py:2309(_setName)
    62073    0.110    0.000    0.110    0.00

<pstats.Stats instance at 0x7fabde731fc8>

In [24]:
p.sort_stats('ncalls').print_stats(30)

Mon May 13 13:46:07 2019    cpout3.txt

         6097065 function calls (5914437 primitive calls) in 4.016 seconds

   Ordered by: call count
   List reduced from 332 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1047852    0.063    0.000    0.063    0.000 {id}
   267360    0.027    0.000    0.027    0.000 {method 'append' of 'list' objects}
   261798    0.036    0.000    0.036    0.000 {method 'get' of 'dict' objects}
   257535    0.052    0.000    0.054    0.000 {getattr}
   195169    0.045    0.000    0.092    0.000 {isinstance}
177645/630    0.206    0.000    2.067    0.003 /usr/lib/python2.7/copy.py:145(deepcopy)
   158793    0.033    0.000    0.033    0.000 {setattr}
   133245    0.017    0.000    0.017    0.000 {method 'startswith' of 'str' objects}
   121281    0.039    0.000    0.039    0.000 /home/hindle1/.local/lib/python2.7/site-packages/music21/common/weakrefTools.py:18(wrapWeakref)
   109557    0.077    0.000    0

<pstats.Stats instance at 0x7fabde731fc8>

We're manipulating the objects alot with their octaves.

In [25]:
# memoize
def pitches_of_key(mkey):
    if not mkey in __pitches_of_key__:
        __pitches_of_key__[mkey] = _pitches_of_key(mkey)
    return __pitches_of_key__[mkey]

__pitches_octave_mod__ = dict()
def pitch_octave_mod( pitch, octave ):
    mkey = str(pitch)+str(octave)
    #mkey = (pitch,octave)
    if not mkey  in __pitches_of_key__:
        __pitches_octave_mod__[mkey] = inkey.note_octave_mod(pitch, octave)
    return __pitches_octave_mod__[mkey]



# memoize
def _get_pitches_of_key_with_octave(mkey, octave):
    our_pitches = pitches_of_key(mkey)
    our_key_notes = [pitch_octave_mod(note, octave-4) for note in our_pitches]
    return our_key_notes

# memoize
__key_and_octave__ = dict()
def get_pitches_of_key_with_octave(mkey, octave):
    our_key = "%s %s" % (mkey,str(octave))
    if not our_key in __key_and_octave__:
        __key_and_octave__[our_key] = _get_pitches_of_key_with_octave(mkey, octave)
    return __key_and_octave__[our_key]    

def generate_abram_song(song_tuples, mkey, note_mapping, bump_mapping, octave):
    """ returns a list of music21 notes """
    # our_pitches = pitches_of_key(mkey)
    # # print(our_pitches) 
    # # modify the key to the appropriate octave
    # our_key_notes = [pitch_octave_mod(note, octave-4) for note in our_pitches]
    our_key_notes = get_pitches_of_key_with_octave(mkey, octave)
    # now get the note mapping to notes
    our_notes = [our_key_notes[x] for x in note_mapping]
    # process the tuple
    # print song_tuples
    # print bump_mapping
    assert sum([len(t)==2 for t in song_tuples]) == len(song_tuples)
    # print [ (our_notes[n], bump_mapping[b-1]) for n,b in song_tuples ]
    return [ make_note(our_notes[n], bump_mapping[b-1]) for n,b in song_tuples ]



In [26]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 27
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 25
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 29
['Emajor',

[27, 25, 29, 27, 26, 24, 28, 28, 28, 27]

No real gain from that.

In [27]:
cProfile.run('my_main(4)','cpout4.txt')


['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 65


In [28]:
p = pstats.Stats('cpout4.txt')
# p.sort_stats('cumtime').print_stats(25)
p.strip_dirs().sort_stats('cumtime').print_stats(50)

Mon May 13 13:46:34 2019    cpout4.txt

         6249985 function calls (6063639 primitive calls) in 4.027 seconds

   Ordered by: cumulative time
   List reduced from 325 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.027    4.027 <string>:1(<module>)
        1    0.000    0.000    4.027    4.027 <ipython-input-2-02a0ed9aa61e>:140(my_main)
        1    0.000    0.000    4.026    4.026 <ipython-input-2-02a0ed9aa61e>:125(grid_search)
       65    0.001    0.000    4.026    0.062 <ipython-input-2-02a0ed9aa61e>:113(generate_and_save_song)
       65    0.000    0.000    3.138    0.048 <ipython-input-2-02a0ed9aa61e>:107(save_midifile)
       65    0.001    0.000    3.138    0.048 __init__.py:246(write)
       65    0.001    0.000    3.138    0.048 base.py:2529(write)
       65    0.001    0.000    3.121    0.048 subConverters.py:921(write)
       65    0.000    0.000    3.033    0.047 translate.py:237(m

<pstats.Stats instance at 0x7fabdc296488>

# Awful Optimization Trick: Remove a layer of abstraction

Music21 objects are pretty slow.

       65    0.001    0.000    3.138    0.048 __init__.py:246(write)
       65    0.001    0.000    3.138    0.048 base.py:2529(write)
       65    0.001    0.000    3.121    0.048 subConverters.py:921(write)
       65    0.000    0.000    3.033    0.047 translate.py:237(music21ObjectToMidiFile)
       65    0.002    0.000    3.033    0.047 translate.py:1890(streamToMidiFile)
       65    0.005    0.000    3.031    0.047 translate.py:1680(streamHierarchyToMidiTracks)
       65    0.003    0.000    2.130    0.033 translate.py:1646(_prepareStreamForMidi)
    180691/130    0.202    0.000    2.021    0.016 copy.py:145(deepcopy)
       65    0.000    0.000    2.019    0.031 __init__.py:1455(__deepcopy__)
       65    0.012    0.000    1.860    0.029 __init__.py:1393(_deepcopySubclassable)

We're going to drop making music21 objects until we need them in the end.

We're going to convert notes to tuples instead of Music21 objects and make Music21 later.

In [29]:
# state is evil
__key_and_octave__ = dict()
__pitches_of_key__ = dict()
__pitches_octave_mod__ = dict()

# make a tuple
def _make_note( midi_pitch, duration ):
    return (midi_pitch, duration)
def make_note( midi_pitch, duration ):
    return _make_note(midi_pitch, duration)
def _get_pitches_of_key_with_octave(mkey, octave):
    our_pitches = pitches_of_key(mkey)
    our_key_notes = [pitch_octave_mod(note, octave-4) for note in our_pitches]
    # convert to midi
    return [p.midi for p in our_key_notes]
    # return our_key_notes
def note_to_midi(note):
    # if not note in __note_to_midi__:
    #     __note_to_midi__[note] = note.pitch.midi # supposedly this is slow
    # return __note_to_midi__[note]
    return int(note[0])

def song_to_txt(song):
    # return " ".join(["M{:03d}".format(note.midi) for note in song])
    return " ".join(["M{:03d}".format(note_to_midi(note)) for note in song])


In [30]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']


AttributeError: 'tuple' object has no attribute 'pitches'

In [31]:
def make_music21_note( midi21_pitch, duration ):
    new_note = music21.note.Note(midi21_pitch)
    new_note.duration = music21.duration.Duration(duration) # is this right    
    return new_note

def song_to_mid(song):
    stream = music21.stream.Stream()
    for elm in song:
        if isinstance(elm, music21.note.Note):
            #note = music21.note.Note()
            #note.pitches = elm.pitches
            #note.duration = elm.duration
            stream.append(elm)
        else:
            stream.append(make_music21_note(elm[0],elm[1]))
    return stream


In [32]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 24
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 24
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 22
['Emajor',

[24, 24, 22, 24, 22, 21, 20, 21, 24, 25]

In [33]:
cProfile.run('my_main(4)','cpout5.txt')


['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 55


In [34]:
p = pstats.Stats('cpout5.txt')
# p.sort_stats('cumtime').print_stats(25)
p.sort_stats('cumtime').print_stats(50)

Mon May 13 13:48:15 2019    cpout5.txt

         6108455 function calls (5882130 primitive calls) in 4.009 seconds

   Ordered by: cumulative time
   List reduced from 316 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.009    4.009 <string>:1(<module>)
        1    0.000    0.000    4.009    4.009 <ipython-input-2-02a0ed9aa61e>:140(my_main)
        1    0.001    0.001    4.009    4.009 <ipython-input-2-02a0ed9aa61e>:125(grid_search)
       55    0.001    0.000    4.008    0.073 <ipython-input-2-02a0ed9aa61e>:113(generate_and_save_song)
       55    0.000    0.000    3.354    0.061 <ipython-input-2-02a0ed9aa61e>:107(save_midifile)
       55    0.000    0.000    3.354    0.061 /home/hindle1/.local/lib/python2.7/site-packages/music21/stream/__init__.py:246(write)
       55    0.001    0.000    3.354    0.061 /home/hindle1/.local/lib/python2.7/site-packages/music21/base.py:2529(write)
       55    0.00

<pstats.Stats instance at 0x7fabde686cb0>

# It's still the midi output.

       55    0.000    0.000    3.354    0.061 <ipython-input-2-02a0ed9aa61e>:107(save_midifile)

Let's try it without without midi altogether.

# Imagine we asked our boss if we could just skip producing MIDI files?

If they said yes and changed the requirements we'd make huge gains.

In [35]:
__grid_search_print__ = False

def save_midifile(x,y):
    return None
def song_to_mid(x):
    return None

In [36]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 7542
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 8888
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 9228
['Em

[7542, 8888, 9228, 8955, 8544, 8868, 9119, 9183, 8487, 9198]

In [37]:
cProfile.run('my_main(4)','cpout6.txt')
p = pstats.Stats('cpout6.txt')
p.sort_stats('cumtime').print_stats(50)

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 20785
Mon May 13 13:48:59 2019    cpout6.txt

         9720994 function calls (9717842 primitive calls) in 4.001 seconds

   Ordered by: cumulative time
   List reduced from 226 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.001    4.001 <string>:1(<module>)
        1    0.000    0.000    4.001    4.001 <ipython-input-2-02a0ed9aa61e>:140(my_main)
        1    0.062    0.062    4.000    4.000 <ipython-input-2-02a0ed9aa61e>:125(grid_search)
    20785    0.093    0.000    3.935    0.000 <ipython-input-2-02a0ed9aa61e>:113(generate_and_save_song)
    20785    0.713    0.000    1.631

<pstats.Stats instance at 0x7fabdc5f8290>

  1808295    0.359    0.000    0.516    0.000 <ipython-input-29-d4e378dee91b>:9(make_note)
    
```make_note``` still is too much



In [38]:
# remove the function call to _make_note
def make_note( midi_pitch, duration ):
    return (midi_pitch, duration)
bench()
cProfile.run('my_main(4)','cpout7.txt')
p = pstats.Stats('cpout7.txt')
p.sort_stats('cumtime').print_stats(20)

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 9263
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 9606
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 9702
['Em

<pstats.Stats instance at 0x7fabde78fd40>

# Mild improvement 8800 to 9300 iters.

Can we memoize further? 

In [39]:
# What about memozing it?
from collections import defaultdict
__make_note__ = defaultdict(dict)
def make_note( midi_pitch, duration ):
    if not duration in __make_note__[midi_pitch]:
        __make_note__[midi_pitch][duration] = (midi_pitch, duration)
    return __make_note__[midi_pitch][duration]

bench()
cProfile.run('my_main(4)','cpout8.txt')
p = pstats.Stats('cpout8.txt')
p.sort_stats('cumtime').print_stats(20)

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 8713
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 8743
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 8695
['Em

<pstats.Stats instance at 0x7fabdc7f1d88>

# OK that didn't work, we're back to 8700

    2005872    0.377    0.000    0.377    0.000 <ipython-input-39-a77dbd047ed2>:4(make_note)
        
Yeah let's undo that change.

In [40]:
# nope memoize did not help
def make_note( midi_pitch, duration ):
    return (midi_pitch, duration)


Well that's neat. But maybe we should do something else? Like actually save midi files to disk?

# Ok but our boss didn't let us off the hook. We still need MIDI output.

* I don't want to change music21
* Maybe I can use another library to output midi?

In [41]:
import midi
TEMPO = 120
TICK = 60.0/(4*16.0*180)

def midi_on_off(when, pitch, duration):
    tick_on_when  = int((when)/TICK)
    tick_off_when = int((duration+when)/TICK)
    on  =  midi.NoteOnEvent(tick=tick_on_when,  pitch=pitch, velocity=100)
    off = midi.NoteOffEvent(tick=tick_off_when, pitch=pitch, velocity=100)
    return (when+duration, on, off)

def song_to_mid(song):
    # Instantiate a MIDI Pattern (contains a list of tracks)
    pattern = midi.Pattern()
    # Instantiate a MIDI Track (contains a list of MIDI events)
    track = midi.Track()
    # Append the track to the pattern
    pattern.append(track)
    # set tempo
    track.append(midi.SetTempoEvent(bpm=TEMPO))
    when = 0.0
    for elm in song:
        if isinstance(elm, music21.note.Note):
            (now, on, off) = midi_on_off(when, elm.midi, elm.duration)
        else:
            (now,on,off) = midi_on_off(when,elm[0],elm[1])
        track.append(on)
        track.append(off)
        when = 0.0 # don't need to keep track of time
    # Add the end of track event, append it to the track
    eot = midi.EndOfTrackEvent(tick=1)
    track.append(eot)
    return pattern

def save_midifile(midi_pattern, midi_file):
    midi.write_midifile(midi_file, midi_pattern)



In [42]:
test_midi = song_to_mid([(60,1.0),(64,1.0),(68,1.0),(60,0.25),(64,0.25),(68,0.25)])
print(test_midi)
save_midifile(test_midi,"test.mid")

midi.Pattern(format=1, resolution=220, tracks=\
[midi.Track(\
  [midi.SetTempoEvent(tick=0, data=[7, 161, 32]),
   midi.NoteOnEvent(tick=0, channel=0, data=[60, 100]),
   midi.NoteOffEvent(tick=192, channel=0, data=[60, 100]),
   midi.NoteOnEvent(tick=0, channel=0, data=[64, 100]),
   midi.NoteOffEvent(tick=192, channel=0, data=[64, 100]),
   midi.NoteOnEvent(tick=0, channel=0, data=[68, 100]),
   midi.NoteOffEvent(tick=192, channel=0, data=[68, 100]),
   midi.NoteOnEvent(tick=0, channel=0, data=[60, 100]),
   midi.NoteOffEvent(tick=48, channel=0, data=[60, 100]),
   midi.NoteOnEvent(tick=0, channel=0, data=[64, 100]),
   midi.NoteOffEvent(tick=48, channel=0, data=[64, 100]),
   midi.NoteOnEvent(tick=0, channel=0, data=[68, 100]),
   midi.NoteOffEvent(tick=48, channel=0, data=[68, 100]),
   midi.EndOfTrackEvent(tick=1, data=[])])])


In [43]:
! xdg-open test.mid

In [44]:
res = bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 940
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 856
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 997
['Emajo

In [45]:
cProfile.run('my_main(4)','cpout9.txt')
p = pstats.Stats('cpout9.txt')
p.sort_stats('cumtime').print_stats(20)

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 2332
Mon May 13 13:53:20 2019    cpout9.txt

         8787843 function calls in 4.002 seconds

   Ordered by: cumulative time
   List reduced from 70 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.002    4.002 <string>:1(<module>)
        1    0.000    0.000    4.002    4.002 <ipython-input-2-02a0ed9aa61e>:140(my_main)
        1    0.045    0.045    4.001    4.001 <ipython-input-2-02a0ed9aa61e>:125(grid_search)
     2332    0.015    0.000    3.956    0.002 <ipython-input-2-02a0ed9aa61e>:113(generate_and_save_song)
     2332    0.206    0.000    2.012    0.001 <ipython-input-41-

<pstats.Stats instance at 0x7fabdc2bf5a8>

# So making MIDIs is still slow

     2332    0.206    0.000    2.012    0.001 <ipython-input-41-fa33ae1b567f>:12(song_to_mid)
   202884    0.347    0.000    1.711    0.000 <ipython-input-41-fa33ae1b567f>:5(midi_on_off)
     2332    0.040    0.000    1.510    0.001 <ipython-input-41-fa33ae1b567f>:35(save_midifile)
    
 

In [50]:
def _midi_on_off(when, pitch, duration):
    tick_on_when  = int((when)/TICK)
    tick_off_when = int((duration+when)/TICK)
    on  =  midi.NoteOnEvent(tick=tick_on_when,  pitch=pitch, velocity=100)
    off = midi.NoteOffEvent(tick=tick_off_when, pitch=pitch, velocity=100)
    return (when+duration, on, off)

__midi_on_off__ = dict()
def midi_on_off(when, pitch, duration):
    t = (when,pitch,duration)
    if not t in __midi_on_off__:
        __midi_on_off__[t] = _midi_on_off(when,pitch,duration)
    return __midi_on_off__[t]

In [51]:
bench()

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 1705
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 1805
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 1849
['Em

[1705, 1805, 1849, 1715, 1856, 1885, 1680, 1674, 1892, 1894]

In [53]:
cProfile.run('my_main(4)','cpout10.txt')
p = pstats.Stats('cpout10.txt')
p.sort_stats('cumtime').print_stats(20)

['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 4206
Mon May 13 13:59:56 2019    cpout10.txt

         9263783 function calls in 4.003 seconds

   Ordered by: cumulative time
   List reduced from 71 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.003    4.003 <string>:1(<module>)
        1    0.000    0.000    4.003    4.003 <ipython-input-2-02a0ed9aa61e>:140(my_main)
        1    0.024    0.024    4.000    4.000 <ipython-input-2-02a0ed9aa61e>:125(grid_search)
     4206    0.026    0.000    3.975    0.001 <ipython-input-2-02a0ed9aa61e>:113(generate_and_save_song)
     4206    0.073    0.000    2.679    0.001 <ipython-input-41

<pstats.Stats instance at 0x7fabdc864200>

# Wow double performance. Great

> Current Performance ~ 1795 iters per 1 seconds 

Midi file saving still big but improved greatly!

In [52]:
# demo?
! xdg-open abram.gen.out/Emajor.O.4.N.7.0.1.2.3.4.5.6.T.1.0_0.5_2.0.mid

# Did we improve?

So from 14-20 iters per second to 1800 per second, doing the same work?

Good. About 80X or 6 orders of magnitude improvement.



# Conclusion

But at what cost?

* Less abstraction
* Less maintainble code
* Less use classes and types
* Less OO

But way faster.
Fundamentally I've simplified the problem and probably should do it in C or C++.

Other ways to improve speed:
* Do I/O in another thread. We're waiting a lot to save results
* Delegate disk I/O to another process
* Avoid disk I/O
* Use a more optimized method of saving
* Replace libraries
* use libs with faster bindings
* Remove a layer of abstraction
* Delegate work elsewhere
* Parallelism

# What is missing:

* Standardized benchmark
* Unit tests (you should always test for correctness)
* Statistic comparison of performance

# BONUS: But what if the improvement is subtle?

If you want to know if you have an improvement you should consider statistical tests:
    
## Student's T-Test

If the data is gaussian/normal/bell curved consider a Student's T-Test to see if your measurements come from the same normal distributions.

## Wilcoxon Rank Sum Test

If the data is not normal Wilcoxon Rank Sum Test can compare the distributions based on rank.

# Interpretting Statistical Tests

Choose an alpha, this is the probability threshold where you accept how many times you want to be wrong to chance. Alpha of 0.05 is common and for optimization is probably ok. Alpha of 0.1 is good for smaller number of runs. Essentially it's a protection against winning the lottery. Sometimes you get lucky.

If you p-value is less than your alpha the tests indicate evidence that your distributions are different. The difference can be negliable but they are statistically significantly different.

We'll use 0.05 as our alpha.

In [56]:
# you might want to run this to make sure the governor is the same
# by the time it runs the rest of the benchmarks
bench() 
res1 = bench()
res2 = bench()



['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 1645
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 1816
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 1882
['Em

Times up! 1885
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 1879
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Times up! 1887
['Emajor', 'B-minor', 'Gminor', 'E-minor', 'G-major', 'Aminor', 'D-major', 'Eminor', 'C-minor', 'Cmajor', 'C#major', 'Bminor', 'D-minor', 'Gmajor', 'F#minor', 'G-minor', 'E-major', 'Dminor', 'Fmajor', 'Cminor', 'C-major', 'B-major', 'Bmajor', 'C#minor', 'F#major', 'A-major', 'Dmajor', 'A-minor', 'Amajor', 'Fminor']
Time

In [57]:
import scipy.stats
import numpy
print("Means: %s %s" % (mean(res1),mean(res2)))
print("Comparison from same distribution")
wc = scipy.stats.wilcoxon( res1, res2 )
print(wc)
print(wc.pvalue <= 0.05)
print("Comparison from 10% worse performance")
wc = scipy.stats.wilcoxon( res1, 0.9*numpy.array(res2) )
print(wc)
print(wc.pvalue <= 0.05)
print("Comparison from same distribution w/ t-test")
tt = scipy.stats.ttest_ind( res1, res2 )
print(tt)
print(tt.pvalue <= 0.05)
print("Comparison from 10% worse performance w/ t-test")
tt = scipy.stats.ttest_ind( res1, 0.9*numpy.array(res2) )
print(tt)
print(tt.pvalue <= 0.05)


Means: 1885 1888
Comparison from same distribution
WilcoxonResult(statistic=15.0, pvalue=0.3742593192802244)
False
Comparison from 10% worse performance
WilcoxonResult(statistic=0.0, pvalue=0.005062032126267864)
True
Comparison from same distribution w/ t-test
Ttest_indResult(statistic=-0.7189195568696245, pvalue=0.48142005814369226)
False
Comparison from 10% worse performance w/ t-test
Ttest_indResult(statistic=47.02979389710427, pvalue=2.7096624555791744e-20)
True


/home/hindle1/.local/lib/python2.7/site-packages/scipy/stats/morestats.py:2388: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")


# BONUS: Do I have to time my own code?

No. ```timeit``` module can do it.

It can do weak reporting of timings. It is best for microbenchmarks.

In [58]:
import timeit
import music21
print(timeit.timeit('music21.note.Note("A#4")',
              setup='import music21',
              number=10000))
print(timeit.timeit('(70,0.25)',
              setup='',
              number=10000))

0.471196889877
0.000195026397705


# Other modules

```benchmark``` provides nicer reporting: https://pypi.org/project/benchmark/
        
Mike's "Python 101: An Intro to Benchmarking your code" https://www.blog.pythonlibrary.org/2016/05/24/python-101-an-intro-to-benchmarking-your-code/